In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
import csv
import re


def pop():
    # 맨 위 뉴스 팝업
    time.sleep(2)
    first = browser.find_element(By.CSS_SELECTOR, '#news-results > div:nth-child(1) > div > div.cont > a > div > strong > span')
    time.sleep(2)
    first.click()

def gettitle(tmp, newsId, newsIdFrom):
    news_title = browser.find_element(By.CSS_SELECTOR, '#news-detail-modal > div > div > div.modal-body > div > div.news-view-head > h1.title')
    print('newsTitle:', news_title.text)
    if (len(news_title.text) < 3):
        print("skip!")
        tmp["newsTitle"].append(news_title.text)
        newsId += 1
        newsIdFrom += 1
        return 1
    tmp["newsTitle"].append(news_title.text)
    return 0

def getbody(tmp):
    # 날짜 저장
    news_date = browser.find_elements(By.XPATH, '//*[@id="news-detail-modal"]/div/div/div[1]/div/div[1]/div[1]/ul/li[1]')
    for value in news_date:
        print('newsDate:',value.text)
        tmp["newsDate"].append(value.text)
    # 하나의 뉴스기사 기준 본문 저장
    news_bodies = browser.find_elements(By.XPATH, '//*[@id="news-detail-modal"]/div/div/div[1]/div/div[2]')
    news_body =''
    for i in news_bodies:
        news_body += i.text
    print('newsBody:',news_body[:50]," ...중략...")
    tmp["newsBody"].append(news_body)

def crawl(newsIdFrom, fromdate, todate):
    # 날짜 구간 설정
    browser.find_element(By.XPATH, '//*[@id="collapse-step-1"]').click()
    time.sleep(2)
    browser.find_element(By.XPATH, '//*[@id="collapse-step-1-body"]/div[3]/div/div[1]/div[1]/a').click()
    time.sleep(2)

    from_ = browser.find_element(By.XPATH, '//*[@id="search-begin-date"]')
    browser.execute_script("arguments[0].value = {};".format(fromdate), from_)
    to_ = browser.find_element(By.XPATH, '//*[@id="search-end-date"]')
    browser.execute_script("arguments[0].value = {};".format(todate), to_)


    # 사건사고 분류 카테고리 - 범죄 선택
    browser.find_element(By.XPATH, '//*[@id="collapse-step-1-body"]/div[3]/div/div[2]/div[3]/a').click()
    time.sleep(2)
    browser.find_element(By.XPATH,'//*[@id="srch-tab4"]/ul/li[1]/div/span[4]').click()
    time.sleep(2)
    browser.find_element(By.XPATH,'//*[@id="search-foot-div"]/div[2]/button[2]').click()
    time.sleep(2)

    # 빈 데이터셋 생성
    tmp = {"newsId":[],"newsTitle":[],"newsDate":[],"newsBody":[]}

    #기간 내 기사를 크롤링
    newsId = 0
    while (newsId < 15000):
        try:
            print('newsId:',newsIdFrom)
            tmp["newsId"].append(newsIdFrom)
            time.sleep(2)
            if (newsId % 10 == 0):
                pop()
            time.sleep(1)
            skip = gettitle(tmp, newsId, newsIdFrom)
            if skip:
                close = browser.find_element(By.CSS_SELECTOR, '#news-detail-modal > div > div > div.modal-body > div > div.list_prev_next > ul > li.nextNewsItem > dd > a > span')
                time.sleep(1)
                close.click()
                time.sleep(1)
                browser.find_element(By.XPATH, '//*[@id="news-results-tab"]/div[1]/div[2]/div/div/div/div/div[4]/a').click()
                newsId = newsId + ( 10 - newsId % 10 )
                newsIdFrom += 1
                continue
            getbody(tmp)
            time.sleep(2)
            # 다음 뉴스 글로 넘기기
            if (newsId == 0):
                browser.find_element(By.CSS_SELECTOR,
                    '#news-detail-modal > div > div > div.modal-body > div > div.list_prev_next > ul > li:nth-child(2) > dd > a > span').click()
            elif (newsId % 10 == 9 or newsId % 10 == 0 ):
                next_news = browser.find_element(By.CSS_SELECTOR, 
                    '#news-detail-modal > div > div > div.modal-body > div > div.list_prev_next > ul > li:nth-child(3) > dd > a > span')
                time.sleep(1)
                next_news.click()
            else:
                next_news = browser.find_element(By.CSS_SELECTOR,
                    '#news-detail-modal > div > div > div.modal-body > div > div.list_prev_next > ul > li.nextNewsItem > dd > a > span')
                time.sleep(1)
                next_news.click()
            newsId += 1
            newsIdFrom += 1
            # save to csv
            df = pd.DataFrame(tmp)
            filename = "dataset_"+fromdate+"_"+todate+".csv"
            if not os.path.exists(filename):
                df.to_csv(filename, index=False, mode="w", encoding="utf-8-sig")
            else:
                df.to_csv(filename, index=False, mode="a", encoding="utf-8-sig", header=False)
        except:
            print("raise exception")
            newsId += 1
            newsIdFrom += 1
        finally:
            if (newsId > 2 and tmp["newsTitle"][-1] == tmp["newsTitle"][-2] == tmp["newsTitle"][-3] 
                and tmp["newsBody"][-1][:10] == tmp["newsBody"][-2][:10] == tmp["newsBody"][-2][:10]):
                break
    # 창 닫기
    time.sleep(2)
    close = browser.find_element(By.CSS_SELECTOR, '#news-detail-modal > div > div > div.modal-footer > button')
    time.sleep(2)
    close.click()
    time.sleep(2)
    
    print("-"*40,"Complete from {} to {}".format(fromdate, todate),"-"*40)
    return newsId


# selenium
browser = webdriver.Chrome()

browser.set_window_size(1920,1000)
url = "https://www.bigkinds.or.kr/v2/news/search.do"

browser.get(url)
time.sleep(2)


newsId = 0

# 기간 설정

# fromdate = "\'2022-04-27\'"
# todate ="\'2022-04-27\'"
# newsId += crawl(newsId, fromdate, todate)


fromdate = "\'2022-04-26\'"
todate ="\'2022-04-26\'"
newsId += crawl(newsId, fromdate, todate)

fromdate = "\'2022-04-25\'"
todate ="\'2022-04-25\'"
newsId += crawl(newsId, fromdate, todate)

fromdate = "\'2022-04-24\'"
todate ="\'2022-04-24\'"
newsId += crawl(newsId, fromdate, todate)

fromdate = "\'2022-04-23\'"
todate ="\'2022-04-23\'"
newsId += crawl(newsId, fromdate, todate)

fromdate = "\'2022-04-22\'"
todate ="\'2022-04-22\'"
newsId += crawl(newsId, fromdate, todate)

print("-"*40,"Complete to acquisite.","-"*40)

newsId: 0
newsTitle: ‘생존수영 교실 보조금 유용’ 광주수영연맹 등 압수수색
newsDate: 2022-04-26
newsBody: 경찰이 광주 생존수영 교실 보조금 유용 의혹 규명을 위해 광주시수영연맹 등 관계기관 압수수  ...중략...
newsId: 1
newsTitle: [지방선거 누가 뛰나-영광군수] 지난 선거 전국 최고 득표율 김준성 3선 도전
newsDate: 2022-04-26
newsBody: 이동권과 경선…승리시 무소속과 대결 강종만 전 군수 무소속 예비후보 등록
김준성
영광군수   ...중략...
newsId: 2
newsTitle: [장석주의 영감과 섬광] 그러니 날씨의 속삭임을 흘려듣지 마라
newsDate: 2022-04-26
newsBody: 날씨는 우리의 원초적 미의 경험이고, 내 안에서 증식하는 무모한 희망이며, 때로는 거쳐야   ...중략...
newsId: 3
newsTitle: [박수진의 논점과 관점] 네거티브 규제, 정권 명운 걸어야
newsDate: 2022-04-26
newsBody: 안철수 대통령직인수위원회 위원장이 그제 규제 시스템을 ‘네거티브 방식’으로 전면 개편하겠다  ...중략...
newsId: 4
newsTitle: [사설] 기존 방송 사업자에만 특혜 보따리…공정과 상식은 어디로
newsDate: 2022-04-26
newsBody: 대통령직인수위원회가 국내 미디어산업 개편 방향에 대한 청사진을 일부 공개했다. 그런데 그   ...중략...
newsId: 5
newsTitle: [한경에세이] 잊혀진 재난 속의 아이들
newsDate: 2022-04-26
newsBody: 지난 한 주간 루마니아를 방문해 우크라이나에서 국경을 넘어온 피란민과 아동들을 만났다. 이  ...중략...
newsId: 6
newsTitle: ‘계곡살인’ 법정공방 예고…보험금 노린 계획범죄vs사고사 과거 판례 어땠나
newsDate: 2022-04-26
newsBody: ‘계곡살인’ 피의자 이은해·조현수 구속심사 ‘계곡

newsTitle: 중앙지검장 "중재안 재고해달라"…OECD도 "우려"
newsDate: 2022-04-26
newsBody: <앵커> 



이에 앞서 검찰은 과거의 잘못을 사과한다면서, 그래도 검찰의 본질적 기능을  ...중략...
newsId: 57
newsTitle: [키워드이슈] 인격권
newsDate: 2022-04-26
newsBody: [KBS 부산]
최근 온·오프라인에서 여러 종류의 인격권 침해 사건이 증가함에 따라 법무부  ...중략...
newsId: 58
newsTitle: [이슈K] 살균제 피해구제 조정안…이대로 무산? vs 극적 합의?
newsDate: 2022-04-26
newsBody: [KBS 전주]
[앵커]

이슈K 시간입니다.

국내 최악의 환경보건 사건으로 기록된 '가  ...중략...
newsId: 59
newsTitle: 스토킹 신고 4배 폭증… 갈길 먼 피해자 보호
newsDate: 2022-04-26
newsBody: 스토킹 피해자 보호에 초점을 맞춘 ‘스토킹 방지 및 피해자 보호 등에 관한 법률‘(스토킹피  ...중략...
newsId: 60
newsTitle: 동물을 학대해도 되는 사회 [박정윤의 으라차차 동물환자]
newsDate: 2022-04-26
newsBody: 경기 화성시 동탄에서 고양이 수십 마리를 학대한 20대 남성을 강력하게 처벌해달라는 국민청  ...중략...
newsId: 61
newsTitle: n번방부터 국정농단 수사까지 막힌다… ‘검수완박’ 4대 논란
newsDate: 2022-04-26
newsBody: 검찰 “수사 막는 ‘범죄 암장법’” 성토 선거범죄 수사 부실화 우려 ‘경찰 권력’ 통제 장  ...중략...
newsId: 62
newsTitle: 인수위 "한국판 넷플릭스 키우겠다"
newsDate: 2022-04-26
newsBody: 박성중 제20대 대통령직인수위원회 과학기술교육분과 간사가 26일 서울 종로구 통의동 인수위  ...중략...
newsId: 63
newsTitle: "

newsId: 112
newsTitle: [오늘의 슈스뉴스] 유재석·미주 소속사 안테나, 악플+루머에 '법적 대응' 예고
newsDate: 2022-04-26
newsBody: 방송인 유재석 씨, 러블리즈 출신 이미주 씨 등의 소속사인 안테나가 악플러와의 전쟁을 예고  ...중략...
newsId: 113
newsTitle: 신종 ‘보이스피싱’…금은방에 입금케 하고 금팔찌로 찾아가
newsDate: 2022-04-26
newsBody: 대전지법 및 고법. 이천열 기자
보이스피싱 범인이 피해자의 돈을 금은방 주인 계좌로 보내게  ...중략...
newsId: 114
newsTitle: '스토킹 피해자 보호법' 정부안 확정… 올해 통과 될까
newsDate: 2022-04-26
newsBody: 한국일보 삽화
작년 10월부터 스토킹 범죄에 대한 처벌법이 시행된 데 이어, 이번엔 스토킹  ...중략...
newsId: 115
newsTitle: 국힘 "검수완박 일방처리시 필리버스터 등 모든 수단 사용"
newsDate: 2022-04-26
newsBody: 국민의힘 의원들이 26일 오후 국회 로텐더홀 계단에서 '검수완박 입법폭주 중단하라' 피켓   ...중략...
newsId: 116
newsTitle: '장제원 아들' 장용준 무면허 음주운전 항소심 6월 9일 시작
newsDate: 2022-04-26
newsBody: 장제원 의원의 아들인 래퍼 노엘(장용준). 사진｜스타투데이DB
국민의힘 장제원 의원의 아들  ...중략...
newsId: 117
newsTitle: 민주당, 검수완박 법안 처리 속도...국민의힘 반발
newsDate: 2022-04-26
newsBody: [앵커]
검수완박 법안 처리를 두고 더불어민주당이 속도를 내면서 재논의를 요구해 온 국민의  ...중략...
newsId: 118
newsTitle: 청주 여중생 유족 '부실수사 원인' 국가 상대 소송 제기
newsDate: 2022-04-26
newsBody: "수사 당시자료 제출…사건 실체 파

newsId: 167
newsTitle: 서석대>봄날, 나비와 장미의 향연
newsDate: 2022-04-26
newsBody: 이탈리아에서는 매년 5~6월 사이에 인피오라타는 축제가 열려 전세계인들을 매혹시킨다. 20  ...중략...
newsId: 168
newsTitle: 전북자원봉사센터서 민주당 당원 입당원서 사본 '수두룩'
newsDate: 2022-04-26
newsBody: 전북경찰청 전경. 사진=전북일보 DB
경찰이 횡령 등의 혐의로 전북자원봉사센터를 압수수색한  ...중략...
newsId: 169
newsTitle: 고소취하 안한다는 이유로 연인 폭행한 50대 항소심도 징역 1년 6개월
newsDate: 2022-04-26
newsBody: 전주지법 재판장 내부. 사진=전북일보 DB
자신에게 폭행당한 연인이 고소를 취하하지 않자   ...중략...
newsId: 170
newsTitle: 푸틴 "우크라, 친러 언론인 암살 모의"...누리꾼들 "주작 아냐?"
newsDate: 2022-04-26
newsBody: 사진-트위터
[머니투데이 윤세미 기자] 블라디미르 푸틴 러시아 대통령이 러시아의 언론인에   ...중략...
newsId: 171
newsTitle: 동종업계 세입자 안 받아…기존 세입자 권리금 어떻게 되나?
newsDate: 2022-04-26
newsBody: 천안 신부문화거리 상점가 일원이다
건물주가 다른 업종으로 임대 내놨다면 권리금보호 의무위반  ...중략...
newsId: 172
newsTitle: [이앤피] "재생에너지 사업에 힘쏟는 루트에너지 윤태환 대표"
newsDate: 2022-04-26
newsBody: ■ 방송 : YTN 라디오 FM 94.5 (13:00~14:00)
■ 진행 : 김우성 앵커  ...중략...
newsId: 173
newsTitle: “겁에 질려 도망가는데 …아내들 급소만 골라 찌른 그놈 엄벌을” 청원
newsDate: 2022-04-26
newsBody: 지난 13일 오전 0시 14분쯤 천안

newsTitle: 
skip!
raise exception
newsId: 300
newsTitle: 
skip!
raise exception
newsId: 301
newsTitle: 
skip!
raise exception
newsId: 302
newsTitle: 
skip!
raise exception
newsId: 303
newsTitle: 
skip!
raise exception
newsId: 304
newsTitle: 
skip!
raise exception
newsId: 305
newsTitle: 
skip!
raise exception
newsId: 306
newsTitle: 
skip!
raise exception
newsId: 307
newsTitle: 
skip!
raise exception
newsId: 308
newsTitle: 
skip!
raise exception
newsId: 309
newsTitle: 
skip!
raise exception
newsId: 310
newsTitle: 
skip!
raise exception
newsId: 311
newsTitle: 
skip!
raise exception
newsId: 312
newsTitle: 
skip!
raise exception
newsId: 313
newsTitle: 
skip!
raise exception
newsId: 314
newsTitle: 
skip!
raise exception
newsId: 315
newsTitle: 
skip!
raise exception
newsId: 316
newsTitle: 
skip!
raise exception
newsId: 317
newsTitle: 
skip!
raise exception
newsId: 318
newsTitle: 
skip!
raise exception
newsId: 319
newsTitle: 
skip!
raise exception
newsId: 320
newsTitle: 
skip!
raise exception


...

newsTitle: 
skip!
raise exception
newsId: 657
newsTitle: 
skip!
raise exception
newsId: 658
newsTitle: 
skip!
raise exception
newsId: 659
newsTitle: 
skip!
raise exception
newsId: 660
newsTitle: 
skip!
raise exception
newsId: 661
newsTitle: 
skip!
raise exception
newsId: 662
newsTitle: 
skip!
raise exception
newsId: 663
newsTitle: 
skip!
raise exception
newsId: 664
newsTitle: 
skip!
raise exception
newsId: 665
newsTitle: 
skip!
raise exception
newsId: 666
newsTitle: 
skip!
raise exception
newsId: 667
newsTitle: 
skip!
raise exception
newsId: 668
newsTitle: 
skip!
raise exception
newsId: 669
newsTitle: 
skip!
raise exception
newsId: 670
newsTitle: 
skip!
raise exception
newsId: 671
newsTitle: 
skip!
raise exception
newsId: 672
newsTitle: 
skip!
raise exception
newsId: 673
newsTitle: 
skip!
raise exception
newsId: 674
newsTitle: 
skip!
raise exception
newsId: 675
newsTitle: 
skip!
raise exception
newsId: 676
newsTitle: 
skip!
raise exception
newsId: 677
newsTitle: 
skip!
raise exception


raise exception
newsId: 835
newsTitle: 
skip!
raise exception
newsId: 836
newsTitle: 
skip!
raise exception
newsId: 837
newsTitle: 
skip!
raise exception
newsId: 838
newsTitle: 
skip!
raise exception
newsId: 839
newsTitle: 
skip!
raise exception
newsId: 840
newsTitle: 
skip!
raise exception
newsId: 841
newsTitle: 
skip!
raise exception
newsId: 842
newsTitle: 
skip!
raise exception
newsId: 843
newsTitle: 
skip!
raise exception
newsId: 844
newsTitle: 
skip!
raise exception
newsId: 845
newsTitle: 
skip!
raise exception
newsId: 846
newsTitle: 
skip!
raise exception
newsId: 847
newsTitle: 
skip!
raise exception
newsId: 848
newsTitle: 
skip!
raise exception
newsId: 849
newsTitle: 
skip!
raise exception
newsId: 850
newsTitle: 
skip!
raise exception
newsId: 851
newsTitle: 
skip!
raise exception
newsId: 852
newsTitle: 
skip!
raise exception
newsId: 853
newsTitle: 
skip!
raise exception
newsId: 854
newsTitle: 
skip!
raise exception
newsId: 855
newsTitle: 
skip!
raise exception
newsId: 856
newsTi

raise exception
newsId: 1013
newsTitle: 
skip!
raise exception
newsId: 1014
newsTitle: 
skip!
raise exception
newsId: 1015
newsTitle: 
skip!
raise exception
newsId: 1016
newsTitle: 
skip!
raise exception
newsId: 1017
newsTitle: 
skip!
raise exception
newsId: 1018
newsTitle: 
skip!
raise exception
newsId: 1019
newsTitle: 
skip!
raise exception
newsId: 1020
newsTitle: 
skip!
raise exception
newsId: 1021
newsTitle: 
skip!
raise exception
newsId: 1022
newsTitle: 
skip!
raise exception
newsId: 1023
newsTitle: 
skip!
raise exception
newsId: 1024
newsTitle: 
skip!
raise exception
newsId: 1025
newsTitle: 
skip!
raise exception
newsId: 1026
newsTitle: 
skip!
raise exception
newsId: 1027
newsTitle: 
skip!
raise exception
newsId: 1028
newsTitle: 
skip!
raise exception
newsId: 1029
newsTitle: 
skip!
raise exception
newsId: 1030
newsTitle: 
skip!
raise exception
newsId: 1031
newsTitle: 
skip!
raise exception
newsId: 1032
newsTitle: 
skip!
raise exception
newsId: 1033
newsTitle: 
skip!
raise excepti

newsTitle: 
skip!
raise exception
newsId: 1188
newsTitle: 
skip!
raise exception
newsId: 1189
newsTitle: 
skip!
raise exception
newsId: 1190
newsTitle: 
skip!
raise exception
newsId: 1191
newsTitle: 
skip!
raise exception
newsId: 1192
newsTitle: 
skip!
raise exception
newsId: 1193
newsTitle: 
skip!
raise exception
newsId: 1194
newsTitle: 
skip!
raise exception
newsId: 1195
newsTitle: 
skip!
raise exception
newsId: 1196
newsTitle: 
skip!
raise exception
newsId: 1197
newsTitle: 
skip!
raise exception
newsId: 1198
newsTitle: 
skip!
raise exception
newsId: 1199
newsTitle: 
skip!
raise exception
newsId: 1200
newsTitle: 
skip!
raise exception
newsId: 1201
newsTitle: 
skip!
raise exception
newsId: 1202
newsTitle: 
skip!
raise exception
newsId: 1203
newsTitle: 
skip!
raise exception
newsId: 1204
newsTitle: 
skip!
raise exception
newsId: 1205
newsTitle: 
skip!
raise exception
newsId: 1206
newsTitle: 
skip!
raise exception
newsId: 1207
newsTitle: 
skip!
raise exception
newsId: 1208
newsTitle: 
s

raise exception
newsId: 1362
newsTitle: 
skip!
raise exception
newsId: 1363
newsTitle: 
skip!
raise exception
newsId: 1364
newsTitle: 
skip!
raise exception
newsId: 1365
newsTitle: 
skip!
raise exception
newsId: 1366
newsTitle: 
skip!
raise exception
newsId: 1367
newsTitle: 
skip!
raise exception
newsId: 1368
newsTitle: 
skip!
raise exception
newsId: 1369
newsTitle: 
skip!
raise exception
newsId: 1370
newsTitle: 
skip!
raise exception
newsId: 1371
newsTitle: 
skip!
raise exception
newsId: 1372
newsTitle: 
skip!
raise exception
newsId: 1373
newsTitle: 
skip!
raise exception
newsId: 1374
newsTitle: 
skip!
raise exception
newsId: 1375
newsTitle: 
skip!
raise exception
newsId: 1376
newsTitle: 
skip!
raise exception
newsId: 1377
newsTitle: 
skip!
raise exception
newsId: 1378
newsTitle: 
skip!
raise exception
newsId: 1379
newsTitle: 
skip!
raise exception
newsId: 1380
newsTitle: 
skip!
raise exception
newsId: 1381
newsTitle: 
skip!
raise exception
newsId: 1382
newsTitle: 
skip!
raise excepti